In [1]:
import os
import sys

NOTEBOOKS_DIR = os.getcwd()
REPO_DIR = os.path.dirname(os.path.abspath(NOTEBOOKS_DIR))
DATA_INGESTION_PACKEGE_DIR = os.path.join(REPO_DIR, "data_ingestion")

sys.path.append(DATA_INGESTION_PACKEGE_DIR)

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mongodb import MongoDBAtlasVectorSearch

# from langchain_core.runnables import RunnablePassthrough
from embedding import EmbeddingClient